In [ ]:
from olap import *

In [ ]:
tmp = generate_rows(10_000)
arrow_array = dump_rows(tmp)
recovered = parse_rows(arrow_array)
recovered['time'] = pd.to_datetime(recovered['timestamps'], unit='s')

# Using persistent store will be identical to using this
# db = ukv.DataBase()
# measurements = db.main
measurements: dict[int, bytes] = {}
cudf_df = cudf.from_pandas(recovered)
cux_df = cux.DataFrame.from_dataframe(cudf_df)

In [ ]:
MAPBOX_API_KEY = "pk.eyJ1IjoiYXRob3J2ZSIsImEiOiJjazBmcmlhYzAwYXc2M25wMnE3bGttbzczIn0.JCDQliLc-XTU52iKa8L8-Q"

chart_map = charts.choropleth(
    x='zips',
    elevation_column='humans',
    elevation_aggregate_fn='mean',
    color_column='amounts',
    color_aggregate_fn='mean',
    mapbox_api_key=MAPBOX_API_KEY,
    geoJSONSource=zip_codes_geojson_url,
    title='ATM Withdrawals Across the US',
)

chart_humans = charts.bokeh.bar('humans')
slider_time = charts.date_range_slider(
    'time', 
    title='Timeframe',
)
overall_amounts = charts.number(
    x='amounts', 
    aggregate_fn='sum', 
    format='${value:,.1f}', 
    title='Total Withdrawals',
)
overall_humans = charts.number(
    x='humans', 
    aggregate_fn='sum', 
    format='{value:,.0f}', 
    title='Humans Detected',
)

d = cux_df.dashboard(
    [chart_map],
    layout_array=[[1]],
    sidebar=[slider_time, overall_amounts, overall_humans],
    theme=cux.themes.rapids,
    title='Map of ATMs',
)

d.app()